<a href="https://colab.research.google.com/github/hdauphin/PHYS420Fall25/blob/main/HW04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 04: Integration
### PHYS420 — Intro to Computational Physics — Fall 2025  
### Hayden Dauphin

![Q1](Images/hw4q1.png)
![Q1.2](Images/hw4q1.2.png)
![Q1.3](Images/hw4q1.3.png)

In [64]:
# HW 4 - Integration

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

#  Calculate an integral using one of the Newton-Cotes rules (n>=2), such as Simpson's rule or higher

def simpson(f, a, b, n):
  """
  Integrate function from a to b using Simpson's rule
  Args:
    f: function
    a: lower limit
    b: upper limit
    n: number of intervals
  Returns:
    Estimate of integral
  """

  # exceptions
  if n%2!=0:  # ensure n is even
    n = n+1

  if a==b:  # return 0 if a equals b
    return 0

  if a>b:  # flip limits and negate result if a>b
    return -1*simpson(f, b, a, n)

  # body of function
  h = ((b-a)/n) # width of intervals
  y0 = f(a)
  yn = f(b)
  x_i = np.linspace(a, b, n+1)
  f_i = np.array([f(x) for x in x_i]) 
  endpoints = y0 + yn
  oddSum = np.sum(f_i[1:n:2])
  evenSum = np.sum(f_i[2:n-1:2])
  return (h/3)*(endpoints + 4*oddSum + 2*evenSum)

# Test integration with three functions 

def f1(x):
  return x**3

def f2(x):
  return np.e ** x

def f3(x):
  return np.sin(x)

estimate1 = simpson(f1, 0, 1, 100)  # analytical answer = 1/4
estimate2 = simpson(f2, 0, 1, 100)  # analytical answer = e - 1
estimate3 = simpson(f3, 0, np.pi, 100)  # analytical answer = 2

secondestimate1 = simpson(f1, 0, 1, 200) 
secondestimate2 = simpson(f2, 0, 1, 200)
secondestimate3 = simpson(f3, 0, np.pi, 200)


def simpsonError(f, initEst, secondEst):
  improved = secondEst + ((secondEst - initEst)/15)
  diff = abs(secondEst-initEst)
  error = diff/15 
  print(f"\t>Improved estimate: ", improved, "\tEstimated error: ", error)
  return error


print("Simpson integral of x^3 (f1) from 0 to 1: ", estimate1)
simpsonError(f1, estimate1, secondestimate1)

print("\nSimpson integral of e^x (f2) from 0 to 1: " , estimate2)
simpsonError(f2, estimate2, secondestimate2)

print("\nSimpson integral of sin(x) (f3) from 0 to pi: ", estimate3)
simpsonError(f3, estimate3, secondestimate3)


# Adaptive integration —— SciPy quad 

adapt1, error1 = quad(f1, 0, 1)
print(f"\nAdaptive integration of x^3 from 0 to 1: ", adapt1, "Estimated error: ", error1)
adapt2, error2 = quad(f2, 0, 1)
print(f"Adaptive integratioin of e^x from 0 to 1: ", adapt2, "Estimated error: ", error2)
adapt3, error3 = quad(f3, 0, np.pi)
print(f"Adaptive integration of sin(x) from 0 to pi: ", adapt3, "Estimated error:", error3)


Simpson integral of x^3 (f1) from 0 to 1:  0.25
	>Improved estimate:  0.25 	Estimated error:  0.0

Simpson integral of e^x (f2) from 0 to 1:  1.7182818285545043
	>Improved estimate:  1.7182818284590458 	Estimated error:  5.966160898651651e-12

Simpson integral of sin(x) (f3) from 0 to pi:  2.0000000108245044
	>Improved estimate:  1.9999999999999367 	Estimated error:  6.765354794898333e-10

Adaptive integration of x^3 from 0 to 1:  0.25 Estimated error:  2.7755575615628914e-15
Adaptive integratioin of e^x from 0 to 1:  1.718281828459045 Estimated error:  1.9076760487502454e-14
Adaptive integration of sin(x) from 0 to pi:  2.0 Estimated error: 2.220446049250313e-14


![Q2](Images/hw4q2.png)

In [76]:
# Rewrite simpson function to handle singularities (2d and 2e)

def simpson(f, a, b, n, avoid_singularity=False, which_end="both", tol_eps=1e-10, max_refine=6):
    """ 
    Integrate function from a to b using Simpson's rule, avoiding endpoint singularities by slicing a tiny epsilon from a and/or b
    Args: 
        f: function 
        a: lower bound 
        b: upper bound 
        n: number of intervals 
        avoid_singularity: whether or not epsilon slicing should be used around singularities
        which_end: "left" = a, "right" = b, "both" = both 
    Returns: 
        Estimate of integral of the function 
    """

    # ensure n is even
    if n % 2 != 0:
        n += 1

    # check that a=/=b
    if a==b:
        return 0
    
    # flip and negate endpoints if a>b
    if a > b: 
        return -1*simpson(f, b, a, n, avoid_singularity, which_end, tol_eps, max_refine)
    
    # Singularity avoidance: 
    if avoid_singularity:
        stepsize = (b - a)/n
        eps0 = max(np.nextafter(a, b) -a, 1e-8 * abs(b - a), 0.25 * stepsize)
        prev = None
        for k in range(max_refine):
            eps_k = eps0/ (10**k)
            a_k, b_k = a, b
            if which_end in ("left", "both"):
                a_k = a + eps_k
            if which_end in ("right", "both"):
                b_k = b - eps_k
            n_k = max(2, int(round((b_k - a_k) / stepsize)))
            if n_k % 2 != 0: 
                n_k += 1 
            I_k = simpson(f, a_k, b_k, n_k)
            if prev is not None and abs(I_k - prev) < tol_eps:
                return I_k
            prev = I_k
        return I_k 
    
    # Body of simpson's rule (same as before):
    h = (b - a) / n
    x_i = np.linspace(a, b, n + 1)
    f_i = np.array([f(x) for x in x_i])
    endpoints = f_i[0] + f_i[-1]
    oddSum = np.sum(f_i[1:n:2])
    evenSum = np.sum(f_i[2:n-1:2])
    return (h / 3) * (endpoints + 4 * oddSum + 2 * evenSum)
        


def f2a(x): 
    return x*(np.e**x)

def f2b(x):
    return 1/(1-0.998*(x**2))

def f2c(x):
    return x*np.sin(12*x)*np.cos(24*x)

def f2d(x): 
    if x==0:
        return 
    return x/((np.e**x) -1)

def f2e(x):
    return x*np.sin(1/x)

# f2a: 
print("Integral of xe^x from 0 to 1:" \
"\nExpected: 1")
f2a1 = simpson(f2a, 0, 1, 100)
f2a2 = simpson(f2a, 0, 1, 200)
print("\t>Simpson integral: ", f2a1)
f2aError = simpsonError(f2a, f2a1, f2a2)
f2aquad, f2aquadError = quad(f2a, 0, 1)
print("\t>Adaptive integration: ", f2aquad, "\tEstimated error: ", f2aquadError)

#f2b:
print("Integral of 1/(1-0.998x^2) from 0 to 1:" \
    "\nExpected: 3.80376")
f2b1 = simpson(f2b, 0, 1, 100)
f2b2 = simpson(f2b, 0, 1, 200)
print("\t>Simspon integral: ", f2b1)
f2bError = simpsonError(f2b, f2b1, f2b2)
f2bquad, f2bquadError = quad(f2b, 0, 1)
print("\t>Adaptive integration: ", f2bquad, "\tEstimated error: ", f2bquadError)

#f2c: 
print("Integral of xsin(12x)cos(24x) from 0 to 2pi:"\
      "\nExpected: 0.17453")
f2c1 = simpson(f2c, 0, 2*np.pi, 100)
f2c2 = simpson(f2c, 0, 2*np.pi, 200)
print("\t>Simspon integral: ", f2c1)
f2cError = simpsonError(f2c, f2c1, f2c2)
f2cquad, f2cquadError = quad(f2c, 0, 1)
print("\t>Adaptive integration: ", f2cquad, "\tEstimated error: ", f2cquadError)

# #f2d: 
print("Integral of x/(e^x-1) from 0 to 1:"\
      "\nExpected: 0.777505")
f2d1 = simpson(f2d, 0, 1, 100, avoid_singularity=True, which_end="left")
f2d2 = simpson(f2d, 0, 1, 200, avoid_singularity=True, which_end="left")
print("\t>Simspon integral: ", f2d1)
f2dError = simpsonError(f2d, f2d1, f2d2)
f2dquad, f2dquadError = quad(f2d, 0, 1)
print("\t>Adaptive integration: ", f2dquad, "\tEstimated error: ", f2dquadError)


#f2e: 
print("Integral of xsin(1/x) from 0 to 1:"\
      "\nExpected: 0.37853")
f2e1 = simpson(f2e, 0, 1, 100, avoid_singularity=True, which_end="left")
f2e2 = simpson(f2e, 0, 1, 200, avoid_singularity=True, which_end="left")
print("\t>Simspon integral: ", f2e1)
f2eError = simpsonError(f2e, f2e1, f2e2)
f2equad, f2equadError = quad(f2e, 0, 1, limit=400)  #increased limit to 400 to accommodate erratic behavior of function
print("\t>Adaptive integration: ", f2equad, "\tEstimated error: ", f2equadError)




Integral of xe^x from 0 to 1:
Expected: 1
	>Simpson integral:  1.0000000004373886
	>Improved estimate:  1.0000000000000004 	Estimated error:  2.733675508181932e-11
	>Adaptive integration:  1.0 	Estimated error:  1.1102230246251565e-14
Integral of 1/(1-0.998x^2) from 0 to 1:
Expected: 3.80376
	>Simspon integral:  4.633139993416674
	>Improved estimate:  4.03193601653446 	Estimated error:  0.03757524855513837
	>Adaptive integration:  3.803756514650994 	Estimated error:  1.2978094445731682e-09
Integral of xsin(12x)cos(24x) from 0 to 2pi:
Expected: 0.17453
	>Simspon integral:  0.14594717185056336
	>Improved estimate:  0.17547569380279426 	Estimated error:  0.0018455326220144324
	>Adaptive integration:  0.03841832476012983 	Estimated error:  3.0651992073270785e-15
Integral of x/(e^x-1) from 0 to 1:
Expected: 0.777505
	>Simspon integral:  0.7775046091133155
	>Improved estimate:  0.7775046224357366 	Estimated error:  8.326513182647469e-10
	>Adaptive integration:  0.7775046341122485 	Estimated 